In [7]:
! pip install google-cloud-automl

In [8]:
! pip install --upgrade google-cloud-storage

Requirement already up-to-date: google-cloud-storage in /srv/app/venv/lib/python3.6/site-packages
Requirement already up-to-date: google-cloud-core<2.0dev,>=1.0.0 in /srv/app/venv/lib/python3.6/site-packages (from google-cloud-storage)
Requirement already up-to-date: google-auth>=1.2.0 in /srv/app/venv/lib/python3.6/site-packages (from google-cloud-storage)
Requirement already up-to-date: google-api-core<2.0.0dev,>=1.6.0 in /srv/app/venv/lib/python3.6/site-packages (from google-cloud-storage)
Requirement already up-to-date: google-resumable-media>=0.3.1 in /srv/app/venv/lib/python3.6/site-packages (from google-cloud-storage)
Requirement already up-to-date: pyasn1-modules>=0.2.1 in /srv/app/venv/lib/python3.6/site-packages (from google-auth>=1.2.0->google-cloud-storage)
Requirement already up-to-date: six>=1.9.0 in /srv/app/venv/lib/python3.6/site-packages (from google-auth>=1.2.0->google-cloud-storage)
Requirement already up-to-date: rsa>=3.1.4 in /srv/app/venv/lib/python3.6/site-packa

In [9]:
%env GOOGLE_APPLICATION_CREDENTIALS key.json

env: GOOGLE_APPLICATION_CREDENTIALS=key.json


In [10]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from matplotlib import style
import re
import difflib as dl
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
import sys

from google.cloud import automl_v1beta1
from google.cloud.automl_v1beta1.proto import service_pb2

In [7]:
def get_prediction(content, project_id, model_id):
	prediction_client = automl_v1beta1.PredictionServiceClient()
	name = 'projects/{}/locations/us-central1/models/{}'.format(project_id, model_id)
	payload = {'text_snippet': {'content': content, 'mime_type': 'text/plain' }}
	params = {}
	request = prediction_client.predict(name, payload, params)
	return request  # waits till request is returned

In [25]:
test_cases = pd.read_csv('test_set_google_nlp_270519.csv')
test_out = pd.read_csv('test_set_google_nlp_270519.csv')
len(test_cases)


284

In [28]:
test_cases = pd.read_csv('test_set_google_nlp_270519.csv')
for i in range(0, len(test_cases)):
    pred = get_prediction(test_cases['Path Report'][i], 'nlpbc3', 'TCN30883917689765005')
    first = True
    for j in pred.payload:
        score = j.classification.score
        cat = j.display_name
        if first:
            test_out['Scored Labels'][i] = cat
        first=False
        test_out[cat][i] = score

test_out

/srv/app/venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/srv/app/venv/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,Path Report,benign_ papilloma_ na,atypical_ fibroadenoma_ na,benign_ usual ductal hyperplasia,benign_ apocrine metaplasia_ na,breast cancer_ na_ na,benign_ sclerosing adenosis_ na,benign_ radial scar_ na,breast cancer_ na_ ductal,atypical_ atypical ductal hyperp,...,benign_ cyst_ na,benign_ pseudoangiomatous stroma,atypical_ flat epithelial atypia,atypical_ atypical lobular hyper,breast cancer_ invasive_ na,breast cancer_ metastasis_ na,breast cancer_ invasive_ lobular,breast cancer_ in situ_ ductal,breast cancer_ invasive_ ductal,Scored Labels
0,Benign cyst see comment,1.034968e-05,3.471667e-08,9.165800e-08,2.851010e-05,6.448144e-05,1.836806e-07,1.957036e-04,1.848819e-06,1.724253e-09,...,9.996973e-01,3.251218e-08,1.418457e-08,3.987254e-08,5.123428e-08,1.211581e-06,2.090936e-08,2.613161e-11,1.561851e-08,benign_ cyst_ na
1,"Fibroadenoma, see comment",2.404561e-05,9.999686e-01,9.549953e-10,2.046292e-11,7.302609e-06,5.681470e-12,4.454057e-08,1.454734e-08,2.033121e-09,...,1.121022e-10,2.858394e-11,4.657135e-10,4.618914e-12,1.051853e-12,5.536685e-12,1.525984e-11,4.726422e-11,5.737162e-11,atypical_ fibroadenoma_ na
2,1 Microscopic focus of invasive ductal carcino...,2.285861e-09,2.448010e-09,1.633331e-10,5.373926e-10,2.328723e-04,3.281350e-11,4.032061e-10,6.376783e-07,2.257651e-07,...,1.632336e-12,3.635001e-10,2.005414e-06,1.112530e-08,2.431302e-03,5.174972e-03,8.011653e-04,6.150779e-03,9.852059e-01,breast cancer_ invasive_ ductal
3,Benign cyst fluid,1.445103e-07,1.980740e-10,2.297003e-09,4.033600e-05,2.478706e-06,2.394605e-08,1.647696e-04,6.429329e-08,1.289009e-11,...,9.997922e-01,1.499954e-09,1.400452e-09,2.455363e-09,5.414600e-10,6.881119e-08,6.722788e-10,1.911046e-13,3.661829e-10,benign_ cyst_ na
4,Benign cyst see comment,1.034968e-05,3.471667e-08,9.165800e-08,2.851010e-05,6.448144e-05,1.836806e-07,1.957036e-04,1.848819e-06,1.724253e-09,...,9.996973e-01,3.251218e-08,1.418457e-08,3.987254e-08,5.123428e-08,1.211581e-06,2.090936e-08,2.613161e-11,1.561851e-08,benign_ cyst_ na
5,"1 Invasive ductal carcinoma, SBR grade 1 see c...",1.791455e-08,3.219192e-10,2.461551e-09,7.009220e-12,2.987184e-04,2.071460e-12,1.875162e-09,9.287730e-06,4.835673e-08,...,1.114051e-11,4.517342e-09,1.796966e-06,4.404629e-11,1.595615e-05,7.047821e-03,5.278624e-07,1.105465e-05,9.926149e-01,breast cancer_ invasive_ ductal
6,Adenocarcinoma see comment,1.305827e-04,7.664138e-05,6.048238e-07,1.110400e-07,9.989755e-01,1.984717e-06,6.433544e-06,4.109236e-04,4.946023e-07,...,3.222990e-05,1.009648e-06,4.825894e-07,1.484897e-07,1.797226e-04,1.658725e-04,2.081172e-06,5.145088e-06,7.413818e-06,breast cancer_ na_ na
7,"1 Borderline phyllodes tumor, 8 cm, transected...",4.187934e-06,6.908856e-05,7.252921e-05,1.572727e-05,4.431202e-04,2.546605e-05,3.939661e-04,2.339928e-06,1.264779e-03,...,3.664116e-08,6.722152e-04,9.912472e-01,6.603129e-05,1.422592e-05,1.934942e-05,7.349367e-06,6.299194e-05,1.361472e-04,atypical_ flat epithelial atypia
8,1 Hyalinized fibroadenoma 2 Microcalcification...,2.018460e-08,9.999998e-01,2.228773e-10,5.241571e-12,1.013897e-07,1.738135e-11,3.977429e-08,3.223451e-10,6.247488e-08,...,1.291003e-13,4.367431e-10,4.619532e-09,2.155422e-10,8.878902e-14,4.467559e-13,9.830665e-12,5.279249e-11,4.483149e-12,atypical_ fibroadenoma_ na
9,1 No carcinoma 2 Radial scar 3 Duct ectasia 4 ...,1.965132e-04,2.059571e-03,4.788398e-02,1.061354e-01,5.190347e-04,6.728718e-02,3.784873e-01,5.355617e-05,1.797170e-02,...,9.734778e-04,5.221455e-02,2.234843e-01,8.128727e-02,1.336777e-04,3.185958e-04,3.367230e-03,1.993105e-04,1.846877e-04,benign_ radial scar_ na


In [29]:
test_out.to_csv('google-nlp-results_280519.csv')

Everything below this line is deprecated and should not be used

In [8]:
kl = pd.read_csv('new_outTEST.csv')
kl = kl.drop(kl.columns[0], axis=1)
path_reports_test = kl['Path Report II']

In [13]:

finout = pd.concat([kl, df], sort=False)
finout

,number,Patient ID,Path Report I,Path Report II,Laterality Derived,Organ Derived,Single Label Derived,benign_other_na,atypical_atypical_ductal_hyperpl,atypical_atypical_lobular_hyperp,...,benign_usual_ductal_hyperplasia,breast_cancer_in_situ_ductal,breast_cancer_in_situ_lobular,breast_cancer_invasive_ductal,breast_cancer_invasive_lobular,breast_cancer_invasive_na,breast_cancer_metastasis_na,breast_cancer_na_ductal,breast_cancer_na_na,other
0,100.0,48.0,"Breast, Fine Needle Aspiration, Right",Atypical cells present,right,breast,"benign, other, na",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6653.0,3511.0,"Left breast, 10 00, 3 cm from the nipple, mam...","Abnormal, suggestive of adenocarcinoma see c...",left,breast,"breast cancer, na, na",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5403.0,3121.0,"Left upper outer breast, needle core biopsy",1 Cystic dilatation of ducts with thick fibr...,left,breast,"benign, other, na",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4508.0,2648.0,"Right breast, wire-localized partial mastectomy","1 Invasive ductal carcinoma, status post neo...",right,breast,"breast cancer, invasive, ductal",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,419.0,259.0,"Breast, Fine Needle Aspiration","Scant benign ductal cells, see note",na,breast,"benign, other, na",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5863.0,3252.0,"Left Breast, Fine Needle Aspiration",Mammary carcinoma see comment,left,breast,"breast cancer, na, na",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5793.0,3230.0,"Left breast, new inferior margin, excision",1 No residual carcinoma 2 Benign breast ...,left,breast,"benign, other, na",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4211.0,2493.0,"Left breast, at 12 o clock, 2 cm from the nip...","Fibrosis, hemorrhage, fat necrosis, and foreig...",left,breast,"benign, other, na",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1520.0,959.0,"Right breast, lumpectomy","1 Invasive ductal carcinoma, 1 5 cm diamete...",right,breast,"breast cancer, invasive, ductal",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3100.0,1887.0,"Right breast, needle localization partial mast...","1 Invasive lobular carcinoma, grade 2, 0 8 ...",right,breast,"breast cancer, invasive, lobular",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
for i in range(0, finout.shape[0]-1):
    k = get_prediction(finout['Path Report II'].iloc[i], 'nlpbc3', 'TCN30883917689765005')
    for j in k.payload:
        finout[j.display_name].iloc[i] = j.classification.score

KeyError: 'breast cancer_ na_ na'

In [39]:
df = pd.DataFrame(columns=['benign_ papilloma_ na', 
'atypical_ fibroadenoma_ na',
'benign_ usual ductal hyperplasia',
'benign_ apocrine metaplasia_ na',
'breast cancer_ na_ na',
'benign_ sclerosing adenosis_ na',
'benign_ radial scar_ na',
'breast cancer_ na_ ductal',
'atypical_ atypical ductal hyperp',
'breast cancer_ in situ_ lobular',
'benign_ cyst_ na',
'benign_ pseudoangiomatous stroma',
'atypical_ flat epithelial atypia',
'atypical_ atypical lobular hyper',
'breast cancer_ invasive_ na',
'breast cancer_ metastasis_ na',
'breast cancer_ invasive_ lobular',
'breast cancer_ in situ_ ductal',
'breast cancer_ invasive_ ductal', "Scored Labels"])
df

,benign_ papilloma_ na,atypical_ fibroadenoma_ na,benign_ usual ductal hyperplasia,benign_ apocrine metaplasia_ na,breast cancer_ na_ na,benign_ sclerosing adenosis_ na,benign_ radial scar_ na,breast cancer_ na_ ductal,atypical_ atypical ductal hyperp,breast cancer_ in situ_ lobular,benign_ cyst_ na,benign_ pseudoangiomatous stroma,atypical_ flat epithelial atypia,atypical_ atypical lobular hyper,breast cancer_ invasive_ na,breast cancer_ metastasis_ na,breast cancer_ invasive_ lobular,breast cancer_ in situ_ ductal,breast cancer_ invasive_ ductal,Scored Labels


In [2]:
outtest = pd.read_csv('new_outTEST.csv')
outtrain = pd.read_csv('new_outTRAIN3.csv')

In [11]:
outtrain = outtrain.drop(outtrain.columns[0:2], axis=1)

In [12]:
outtrain['Single Label Derived'].value_counts()
outtrain_3_other = outtrain.replace({'breast cancer_in situ_na': 'other', 'lymphoma_na_na': 'other', 
                              'benign_mastitis_na': 'other', 'breast cancer_na_lobular':'other', 'atypical_phyllodes_na':'other'})
outtrain_3_other['Single Label Derived'].value_counts()

benign_other_na                                    300
breast cancer_invasive_ductal                       50
breast cancer_in situ_ductal                        43
breast cancer_na_na                                 35
atypical_fibroadenoma_na                            30
benign_cyst_na                                      18
benign_apocrine metaplasia_na                       18
benign_usual ductal hyperplasia_na                  17
breast cancer_invasive_lobular                      11
atypical_atypical ductal hyperplasia_na              8
benign_sclerosing adenosis_na                        6
other                                                5
breast cancer_in situ_lobular                        5
atypical_atypical lobular hyperplasia_na             5
breast cancer_invasive_na                            4
breast cancer_na_ductal                              4
breast cancer_metastasis_na                          2
benign_papilloma_na                                  1
benign_pse

In [13]:
outtrain_3_removed = outtrain_3_other.drop(outtrain_3_other[outtrain_3_other['Single Label Derived'] == 'other'].index)
outtrain_3_removed['Single Label Derived'].value_counts()

benign_other_na                                    300
breast cancer_invasive_ductal                       50
breast cancer_in situ_ductal                        43
breast cancer_na_na                                 35
atypical_fibroadenoma_na                            30
benign_cyst_na                                      18
benign_apocrine metaplasia_na                       18
benign_usual ductal hyperplasia_na                  17
breast cancer_invasive_lobular                      11
atypical_atypical ductal hyperplasia_na              8
benign_sclerosing adenosis_na                        6
breast cancer_in situ_lobular                        5
atypical_atypical lobular hyperplasia_na             5
breast cancer_invasive_na                            4
breast cancer_na_ductal                              4
breast cancer_metastasis_na                          2
benign_papilloma_na                                  1
benign_pseudoangiomatous stromal hyperplasia_na      1
atypical_f

In [14]:
outtrain_1_other = outtrain.replace({'bresat': 'breast'})
outtrain_1_other['Single Label Derived'] = outtrain_1_other['Single Label Derived'].apply(lambda x: x.split('_')[0])
outtrain_1_other['Single Label Derived'].value_counts()

benign           361
breast cancer    154
atypical          45
lymphoma           4
Name: Single Label Derived, dtype: int64

In [15]:
outtrain_1_removed = outtrain_1_other.drop(outtrain_1_other[outtrain_1_other['Single Label Derived'] == 'lymphoma'].index)
outtrain_1_removed['Single Label Derived'].value_counts()

benign           361
breast cancer    154
atypical          45
Name: Single Label Derived, dtype: int64

In [16]:
outtrain_3_other.to_csv('training_set_group4.csv')
outtrain_3_removed.to_csv('training_set_group3.csv')
outtrain_1_removed.to_csv('training_set_group1.csv')
outtrain_1_other.to_csv('training_set_group2.csv')

In [10]:
kl = pd.read_csv('new_outTEST.csv')
kl = kl.rename(index=str, columns={'Single Label  Derived ': "Single Label Derived"})
kl['Single Label Derived'].replace(regex=True,inplace=True,to_replace=r', ',value=r'_')
kl

,Unnamed: 0,number,Patient ID,Path Report I,Path Report II,Laterality Derived,Organ Derived,Single Label Derived
0,0,100,48,"Breast, Fine Needle Aspiration, Right",Atypical cells present,right,breast,benign_other_na
1,1,6653,3511,"Left breast, 10 00, 3 cm from the nipple, mam...","Abnormal, suggestive of adenocarcinoma see c...",left,breast,breast cancer_na_na
2,2,5403,3121,"Left upper outer breast, needle core biopsy",1 Cystic dilatation of ducts with thick fibr...,left,breast,benign_other_na
3,3,4508,2648,"Right breast, wire-localized partial mastectomy","1 Invasive ductal carcinoma, status post neo...",right,breast,breast cancer_invasive_ductal
4,4,419,259,"Breast, Fine Needle Aspiration","Scant benign ductal cells, see note",na,breast,benign_other_na
5,5,5863,3252,"Left Breast, Fine Needle Aspiration",Mammary carcinoma see comment,left,breast,breast cancer_na_na
6,6,5793,3230,"Left breast, new inferior margin, excision",1 No residual carcinoma 2 Benign breast ...,left,breast,benign_other_na
7,7,4211,2493,"Left breast, at 12 o clock, 2 cm from the nip...","Fibrosis, hemorrhage, fat necrosis, and foreig...",left,breast,benign_other_na
8,8,1520,959,"Right breast, lumpectomy","1 Invasive ductal carcinoma, 1 5 cm diamete...",right,breast,breast cancer_invasive_ductal
9,9,3100,1887,"Right breast, needle localization partial mast...","1 Invasive lobular carcinoma, grade 2, 0 8 ...",right,breast,breast cancer_invasive_lobular
